In [185]:
import psycopg2
from psycopg2 import sql
import re 
import json
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import pandas as pd
from tkinter import Tk
from tkinter import filedialog
import csv
import io
hostname = 'localhost'
database = 'First_test'
username = 'postgres'
pwd = 'Yann'
port_id = 5432

In [186]:
def connect_to_database():
    try:
        conn = psycopg2.connect(
            host=hostname,
            dbname=database,
            user=username,
            password=pwd,
            port=port_id
        )
        return conn
    except Exception as error:
        print(error)
        return None
    


In [187]:
def get_data(limit=None):
    table_name = input("Enter the table name: ")
    while not isinstance(table_name, str):
        print("Table name must be a string.")
        table_name = input("Enter the table name: ")
    conn = connect_to_database()
    if conn:
        try:
            with conn.cursor() as cur:
                if limit:
                    cur.execute(sql.SQL("SELECT * FROM public.{} LIMIT %s;").format(sql.Identifier(table_name)), (limit,))
                else:
                    cur.execute(sql.SQL("SELECT * FROM public.{};").format(sql.Identifier(table_name)))
                rows = cur.fetchall()
                column_names = [desc[0] for desc in cur.description]
                
                df = pd.DataFrame(rows, columns=column_names)
                return df
        except Exception as error:
            print(error)
        finally:
            conn.close()



In [188]:
df = get_data()
    
    # Créer le DataFrame

    
if df is not None:
    print("DataFrame created successfully:")
    print(df.head())
else:
    print("No data available.")

DataFrame created successfully:
   id        date   3__low To_Symbol Digital_Currency_Code 4__close  2__high  \
0   1  2024-02-19  1.06945       USD                  None  1.07804  1.08974   
1   2  2024-01-31  1.07947       USD                  None  1.08180  1.10460   
2   3  2023-12-29  1.07230       USD                  None  1.10374  1.11395   
3   4  2023-11-30  1.05165       USD                  None  1.08861  1.10171   
4   5  2023-10-31  1.04480       USD                  None  1.05744  1.06943   

  From_Symbol Digital_Currency_Name  1__open  
0         EUR                  None  1.08180  
1         EUR                  None  1.10400  
2         EUR                  None  1.08860  
3         EUR                  None  1.05740  
4         EUR                  None  1.05676  


In [189]:
df.dtypes

id                        int64
date                     object
3__low                   object
To_Symbol                object
Digital_Currency_Code    object
4__close                 object
2__high                  object
From_Symbol              object
Digital_Currency_Name    object
1__open                  object
dtype: object

In [190]:
import pandas as pd
import re
import warnings
import numpy as np

warnings.filterwarnings("ignore", message="Could not infer format")

def infer_data_types(df):
    # Define the percentage of rows to sample
    sample_percentage = 0.8
    
    # Sample the data
    sample = df.sample(frac=sample_percentage, random_state=42)
    
    # Dictionary to store inferred data types for each column
    inferred_data_types = {}
    
    # Loop through each column and infer the data type
    for col in sample.columns:
        inferred_data_type = infer_data_type(sample[col])
        inferred_data_types[col] = inferred_data_type
    
    return inferred_data_types

import pandas as pd
import numpy as np

def infer_data_type(series):
    if series.dtype == 'object':
        # Convert non-string values to strings
        series = series.astype(str)
        
        # Replace 'None' with NaN
        series = series.replace('None', np.nan).infer_objects(copy=False)
        
        # Remove empty strings and NaNs from the series
        non_empty_series = series.dropna()
        non_empty_series = non_empty_series[non_empty_series != '']
        # Convert all non-empty series elements to strings
        non_empty_series = non_empty_series.astype(str)
        
        # Check if the series contains mostly integers
        if non_empty_series.str.replace('-', '').str.isdigit().mean() > 0.8:
            return 'datetime.date'
        
        # Check if the series contains mostly floats
        elif non_empty_series.str.replace('.', '').str.replace('-', '').str.replace(',', '').str.replace('%', '').str.isdigit().mean() > 0.8:
            return 'float64'
        
        # Check if the series contains mostly dates
        elif non_empty_series.str.match(r'^\d{4}-\d{2}-\d{2}$').mean() > 0.8:
            return 'datetime.date'
        
        # Check if the series contains mostly currency values ('$XXX.XX')
        elif non_empty_series.str.startswith('$').mean() > 0.8:
            return 'float64'
        
        # Check if the series contains mostly years (4-digit numbers)
        elif non_empty_series.str.match(r'^\d{4}$').mean() > 0.8:
            return 'int64'
        
        # Check if the series contains mostly percentage values ('X.XX%')
        elif non_empty_series.str.match(r'^-?\d+\.\d+%$').mean() > 0.8:
            return 'float64'
        
        # If none of the above conditions are met, assume it's an object
        else:
            return 'object'
    elif series.dtype == 'datetime64[ns]':
        return 'datetime.date'
    else:
        return str(series.dtype)

def convert_columns(df, inferred_types):
    for col, dtype in inferred_types.items():
        if dtype == 'datetime.date':
            df[col] = df[col].astype(str)
            df[col] = df[col].replace('None', np.nan)
            df[col] = df[col].str.replace('-', '')
            df[col] = pd.to_datetime(df[col]).dt.date
        elif dtype == 'int64':
            # Apply cleaning operations similar to infer_data_type function
            df[col] = df[col].astype(str)
            df[col] = df[col].replace('None', np.nan)
            df[col] = df[col].str.replace('-', '')
            df[col] = pd.to_numeric(df[col]).fillna(0).astype(int)   
        elif dtype == 'float64':
            df[col] = df[col].astype(str)
            df[col] = df[col].replace('None', np.nan)
            df[col] = df[col].str.replace('-', '')
            df[col] = df[col].str.replace('.', '').str.replace('-', '').str.replace(',', '').str.replace('%', '').str.replace('$','')
            df[col] = pd.to_numeric(df[col]).fillna(0).astype(float)
        
        elif dtype == 'object':
            df[col] = df[col].astype(str)
        # Add more conditions for other data types if needed
    return df






# Example usage
# Replace df with your actual DataFrame
inferred_types = infer_data_types(df)
print("Inferred data types:")
print(inferred_types)



Inferred data types:
{'id': 'int64', 'date': 'datetime.date', '3__low': 'float64', 'To_Symbol': 'object', 'Digital_Currency_Code': 'object', '4__close': 'float64', '2__high': 'float64', 'From_Symbol': 'object', 'Digital_Currency_Name': 'object', '1__open': 'float64'}


C:\Users\yskon\AppData\Local\Temp\ipykernel_11020\394953751.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  series = series.replace('None', np.nan).infer_objects(copy=False)
C:\Users\yskon\AppData\Local\Temp\ipykernel_11020\394953751.py:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  series = series.replace('None', np.nan).infer_objects(copy=False)


In [191]:
# Assuming 'inferred_types' is a dictionary containing column names and their inferred types
df = convert_columns(df, inferred_types)

# Now df contains columns converted to the inferred data types
print(df.dtypes)

# Corresponding type of row in columns
for col, dtype in inferred_types.items():
    print(f"Column '{col}' corresponds to type '{dtype}'")


id                         int32
date                      object
3__low                   float64
To_Symbol                 object
Digital_Currency_Code     object
4__close                 float64
2__high                  float64
From_Symbol               object
Digital_Currency_Name     object
1__open                  float64
dtype: object
Column 'id' corresponds to type 'int64'
Column 'date' corresponds to type 'datetime.date'
Column '3__low' corresponds to type 'float64'
Column 'To_Symbol' corresponds to type 'object'
Column 'Digital_Currency_Code' corresponds to type 'object'
Column '4__close' corresponds to type 'float64'
Column '2__high' corresponds to type 'float64'
Column 'From_Symbol' corresponds to type 'object'
Column 'Digital_Currency_Name' corresponds to type 'object'
Column '1__open' corresponds to type 'float64'


In [192]:
df.head()

,id,date,3__low,To_Symbol,Digital_Currency_Code,4__close,2__high,From_Symbol,Digital_Currency_Name,1__open
0,1,2024-02-19,106945.0,USD,None,107804.0,108974.0,EUR,None,108180.0
1,2,2024-01-31,107947.0,USD,None,108180.0,110460.0,EUR,None,110400.0
2,3,2023-12-29,107230.0,USD,None,110374.0,111395.0,EUR,None,108860.0
3,4,2023-11-30,105165.0,USD,None,108861.0,110171.0,EUR,None,105740.0
4,5,2023-10-31,104480.0,USD,None,105744.0,106943.0,EUR,None,105676.0
